In [14]:
import pickle
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
import torch
import torch.nn as nn
import pickle
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings
import lightgbm as lgb
from sklearn.preprocessing import MinMaxScaler

warnings.filterwarnings('ignore')

class RiskClassifier:
    def __init__(self):
        self.model = lgb.LGBMClassifier(random_state=42, is_unbalance=True)
        self.features = [
            'open', 'high', 'low', 'close', 'volume',
            'ema_50', 'ema_200', 'macd', 'macd_signal', 'macd_diff', 'rsi',
            'bollinger_hband', 'bollinger_lband', 'mfi',
            'return', 'bollinger_pct', 'bollinger_bw',
            'volatility_5d', 'volatility_10d',
            'volume_change', 'obv', 'volume_pct_20d', 'close_lag1', 'close_lag2',
            'close_lag3', 'return_lag1', 'return_lag2', 'return_lag3', 'rsi_lag1',
            'rsi_lag2', 'rsi_lag3', 'macd_lag1', 'macd_lag2', 'macd_lag3',
            'macd_diff_lag1', 'macd_diff_lag2', 'macd_diff_lag3',
            'bollinger_pct_lag1', 'bollinger_pct_lag2', 'bollinger_pct_lag3'
        ]

    def train(self, df_train_featured):
        df_train = df_train_featured.copy()
        
        test_size = 0.20
        all_Xtrain_dfs = []
        all_ytrain_dfs = []
        

        for ticker in df_train["ticker"].unique():
            df_ticker = df_train[df_train["ticker"] == ticker].copy()
            df_ticker.sort_values(by = "timestamp", inplace = True)
            split_index = int((len(df_ticker) * (1-test_size)))
            df_ticker_train = df_ticker[:split_index]
            all_Xtrain_dfs.append(df_ticker_train[self.features])
            all_ytrain_dfs.append(df_ticker_train["risk_label"])
        

        X_train = pd.concat(all_Xtrain_dfs)
        y_train = pd.concat(all_ytrain_dfs)
         
        self.model.fit(X_train, y_train)
        print("[Risk Model] Huấn luyện xong")

    def predict(self, df_today_featured):
        if df_today_featured.empty:
            return pd.Series(dtype=int)

        
        X_today = df_today_featured[self.features].dropna()
        if X_today.empty:
            return pd.Series(dtype=int)
            
        predictions = self.model.predict(X_today)
        return pd.Series(predictions, index=X_today.index)
    
    def load_model(self, model_path: str):
        """Load model đã được lưu bằng pickle (.pkl)."""
        with open(model_path, "rb") as f:
            self.model = pickle.load(f)
        return self.model

In [15]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_layer_size, num_classes, num_layers=2, dropout=0.3):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers, 
                            batch_first=True, dropout=dropout, bidirectional=True)
        self.linear = nn.Linear(hidden_layer_size * 2, num_classes)

    def forward(self, input_seq):
        device = input_seq.device
        h0 = torch.zeros(self.num_layers * 2, input_seq.size(0), self.hidden_layer_size).to(device)
        c0 = torch.zeros(self.num_layers * 2, input_seq.size(0), self.hidden_layer_size).to(device)
        lstm_out, _ = self.lstm(input_seq, (h0, c0))
        predictions = self.linear(lstm_out[:, -1, :])
        return predictions

In [16]:
class SignalClassifier:
    def __init__(self, model_path, top_features, scaler_path,
                 time_steps=10, device="cpu"):
        self.model_path = model_path
        self.top_features = top_features
        self.scaler_path = scaler_path
        self.time_steps = time_steps
        self.device = device
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        self.model = None

    def load_model(self, input_size, num_classes=3, hidden_size=128):
        with open(self.scaler_path, "rb") as f:
            self.scaler = pickle.load(f)

        model = LSTMClassifier(input_size=input_size, hidden_layer_size=hidden_size, num_classes=num_classes)
        model.load_state_dict(torch.load(self.model_path, map_location=self.device))
        model.to(self.device)
        model.eval()
        self.model = model
        print(f"[Signal Model] Load weight từ {self.model_path} thành công")
        print(f"[Signal Model] Load scaler từ {self.scaler_path} thành công")

    def _create_lstm_dataset(self, X_scaled):
        if len(X_scaled) < self.time_steps:
            return np.array([])
        return X_scaled[-self.time_steps:].reshape(1, self.time_steps, X_scaled.shape[1])

    def predict(self, df_history_featured):
        if df_history_featured.empty or len(df_history_featured) < self.time_steps:
            return 1 

        X_hist = df_history_featured[self.top_features].values
        X_hist_scaled = self.scaler.transform(X_hist)

        X_lstm = self._create_lstm_dataset(X_hist_scaled)
        if X_lstm.shape[0] == 0:
            return 1 
        
        X_tensor = torch.tensor(X_lstm, dtype=torch.float32).to(self.device)

        with torch.no_grad():
            output = self.model(X_tensor)
            pred = torch.argmax(output, dim=1).item()
        
        return pred

In [17]:
def add_features(df):
    df_copy = df.copy()

    # Features cho Risk Model
    df_copy["volatility_5d"] = df_copy.groupby("ticker")["return"].rolling(5).std().reset_index(0, drop=True)
    df_copy["volatility_10d"] = df_copy.groupby("ticker")["return"].rolling(10).std().reset_index(0, drop=True)

    df_copy["volume_change"] = df_copy.groupby("ticker")["volume"].pct_change()
    
    df_copy["obv"] = (np.sign(df_copy["return"].fillna(0)) * df_copy["volume"]).groupby(df_copy["ticker"]).cumsum()
    
    df_copy["volume_pct_20d"] = df_copy.groupby("ticker")["volume"].transform(
        lambda x: x.rolling(20).apply(lambda s: pd.Series(s).rank(pct=True).iloc[-1])
    )

    lag_cols = ["close", "return", "rsi", "macd", "macd_diff", "bollinger_pct"]
    for col in lag_cols:
        for lag in [1, 2, 3]:
            df_copy[f"{col}_lag{lag}"] = df_copy.groupby("ticker")[col].shift(lag)
    
    # Features cho Signal Model
    lags = [5, 10, 15]
    for lag in lags:
        df_copy[f'return_lag_{lag}'] = df_copy.groupby('ticker')['return'].shift(lag)

    vol_windows = [10, 20]
    for window in vol_windows:
        df_copy[f'volatility_{window}'] = df_copy.groupby('ticker')['return'].rolling(window=window).std().reset_index(0, drop=True)

    df_copy['rsi_momentum'] = df_copy.groupby('ticker')['rsi'].diff()
    df_copy['macd_diff_momentum'] = df_copy.groupby('ticker')['macd_diff'].diff()
    df_copy['price_vs_ema200'] = (df_copy['close'] / df_copy['ema_200']) - 1

    indicators_to_lag = ['rsi', 'mfi']
    for indicator in indicators_to_lag:
        for lag in [1, 3]:
            df_copy[f'{indicator}_lag_{lag}'] = df_copy.groupby('ticker')[indicator].shift(lag)

    return df_copy

def calculate_risk_label(df):
    df['min_price_in_future'] = df.groupby('ticker')['low'].shift(-10).rolling(window=10).min()

    df['future_max_drawdown'] = (df['min_price_in_future'] / df["close"]) - 1

    df.dropna(subset=['future_max_drawdown'], inplace=True)

    df['T2_drawdown'] = df.groupby('timestamp')['future_max_drawdown'].transform(lambda x: x.quantile(0.35))

    labels = [1, 0]
    conditions = [
        df['future_max_drawdown'] <= df['T2_drawdown'],
        df['future_max_drawdown'] > df['T2_drawdown']
    ]
    df['risk_label'] = np.select(conditions, labels)

    return df

In [18]:
def create_all_features(df, window_size=252):
    df_copy = df.copy()

    df_vnx = df[df["exchange"] == "HOSE"]
    df_hnx = df[df["exchange"] == "HNX"]
    df_upcom = df[df["exchange"] == "UPCOM"]

    df_vnx = calculate_risk_label(df_vnx)
    df_hnx = calculate_risk_label(df_hnx)
    df_upcom = calculate_risk_label(df_upcom)

    df_vnx = add_features(df_vnx)
    df_hnx = add_features(df_hnx)
    df_upcom = add_features(df_upcom)
    
    return df_vnx, df_hnx, df_upcom

In [19]:
def filter_ticker_by_indicators(data):
    # Ngưỡng chỉ số cho VN-Index
    vn_rsi_low, vn_rsi_high = 30, 70
    vn_macd_low, vn_macd_high = -190.806461, 191.497498
    vn_boll_low, vn_boll_high = 0.247301, 0.769456

    # Ngưỡng chỉ số cho HNX-Index 
    hnx_rsi_low, hnx_rsi_high = 25, 75
    hnx_macd_low, hnx_macd_high = -171.424148, 176.457420
    hnx_boll_low, hnx_boll_high = 0.230735, 0.735965

    # Ngưỡng chỉ số cho UPCOM 
    upcom_rsi_low, upcom_rsi_high = 20, 80
    upcom_macd_low, upcom_macd_high = -161.357600, 163.167889
    upcom_boll_low, upcom_boll_high = 0.224304, 0.733178

    # Lọc cổ phiếu cho từng sàn
    vn_filtered = data[
        (data['exchange'] == 'VN') &
        (data['rsi'] >= vn_rsi_low) & (data['rsi'] <= vn_rsi_high) &
        (data['macd_diff'] >= vn_macd_low) & (data['macd_diff'] <= vn_macd_high) &
        (data['bollinger_pct'] >= vn_boll_low) & (data['bollinger_pct'] <= vn_boll_high)
    ]

    hnx_filtered = data[
        (data['exchange'] == 'HNX') &
        (data['rsi'] >= hnx_rsi_low) & (data['rsi'] <= hnx_rsi_high) &
        (data['macd_diff'] >= hnx_macd_low) & (data['macd_diff'] <= hnx_macd_high) &
        (data['bollinger_pct'] >= hnx_boll_low) & (data['bollinger_pct'] <= hnx_boll_high)
    ]

    upcom_filtered = data[
        (data['exchange'] == 'UPCOM') &
        (data['rsi'] >= upcom_rsi_low) & (data['rsi'] <= upcom_rsi_high) &
        (data['macd_diff'] >= upcom_macd_low) & (data['macd_diff'] <= upcom_macd_high) &
        (data['bollinger_pct'] >= upcom_boll_low) & (data['bollinger_pct'] <= upcom_boll_high)
    ]


    filtered_df = pd.concat([vn_filtered, hnx_filtered, upcom_filtered], ignore_index=True)

    return filtered_df


In [20]:
def filter_ticker(vnx_finance_path, hnx_finance_path, upcom_finance_path):
    list_qualified_tickers = []
    df_vnx = pd.read_excel(vnx_finance_path)
    df_hnx = pd.read_excel(hnx_finance_path)
    df_upcom = pd.read_excel(upcom_finance_path)

    # print(df_hnx.head())
    # print(df_vnx.head())
    # print(df_upcom.head())

    for ticker in df_hnx["ticker"].unique():
        row = df_hnx[df_hnx["ticker"] == ticker].iloc[0]  
        if (
            row["EBITMargin"] >= 0.03
            and row["ROA"] >= 0.01
            and row["ROE"] >= 0.01
            and row["ROIC"] >= 0.02
        ):
            list_qualified_tickers.append(ticker)

    
    for ticker in df_vnx["ticker"].unique():
        row = df_vnx[df_vnx["ticker"] == ticker].iloc[0]  
        if (
            row["EBITMargin"] >= 0.05
            and row["ROA"] >= df_vnx["ROA"].quantile(0.25)
            and row["ROE"] >= df_vnx["ROE"].quantile(0.25)
            and row["ROIC"] >= 0.04
        ):
            list_qualified_tickers.append(ticker)

    for ticker in df_upcom["ticker"].unique():
        row = df_upcom[df_upcom["ticker"] == ticker].iloc[0]  
        if (
            row["EBITMargin"] >= df_upcom["EBITMargin"].quantile(0.35)
            and row["ROA"] >= df_upcom["ROA"].quantile(0.25)
            and row["ROE"] >= df_upcom["ROE"].quantile(0.25)
            and row["ROIC"] >= df_upcom["ROIC"].quantile(0.35)
        ):
            list_qualified_tickers.append(ticker)

    return list_qualified_tickers

VNX_FINANCE_PATH = r"D:\Downloads\DSTC vòng 3\finance data\VNINDEX_Finance.xlsx"
HNX_FINANCE_PATH = r"D:\Downloads\DSTC vòng 3\finance data\HNXINDEX_Finance.xlsx"
UPCOM_FINANCE_PATH = r"D:\Downloads\DSTC vòng 3\finance data\UPCOMINDEX_Finance.xlsx"
list_qualified_tickers = filter_ticker(VNX_FINANCE_PATH, HNX_FINANCE_PATH, UPCOM_FINANCE_PATH)
print(len(list_qualified_tickers))

561


In [21]:
import time
from FiinQuantX import FiinSession, BarDataUpdate
from typing import List
import requests
import pandas as pd

# ===== 1. KẾT NỐI API FIINQUANT =====
username = 'DSTC_28@fiinquant.vn'
password = 'Fiinquant0606'

client = FiinSession(username=username, password=password).login()
fi = client.FiinIndicator()
# ===== 2. CLASS GỬI TELEGRAM =====
class TelegramNotifier:
    def __init__(self, bot_token: str, chat_id: str):
        self.bot_token = bot_token
        self.chat_id = chat_id

    def send_message(self, text: str):
        url = f"https://api.telegram.org/bot{self.bot_token}/sendMessage"
        payload = {
            "chat_id": self.chat_id,
            "text": text,
            "parse_mode": "HTML"
        }
        try:
            r = requests.post(url, data=payload, timeout=5)
            r.raise_for_status()
            print("📩 Gửi Telegram thành công:", r.json())
        except Exception as e:
            print("❌ Lỗi gửi Telegram:", e)
            if 'r' in locals():
                print("Response:", r.text)

In [22]:
def add_features(df):
    df_copy = df.copy()

    # Features cho Risk Model
    df_copy["volatility_5d"] = df_copy.groupby("ticker")["return"].rolling(5).std().reset_index(0, drop=True)
    df_copy["volatility_10d"] = df_copy.groupby("ticker")["return"].rolling(10).std().reset_index(0, drop=True)

    df_copy["volume_change"] = df_copy.groupby("ticker")["volume"].pct_change()
    
    df_copy["obv"] = (np.sign(df_copy["return"].fillna(0)) * df_copy["volume"]).groupby(df_copy["ticker"]).cumsum()
    
    df_copy["volume_pct_20d"] = df_copy.groupby("ticker")["volume"].transform(
        lambda x: x.rolling(20).apply(lambda s: pd.Series(s).rank(pct=True).iloc[-1])
    )

    lag_cols = ["close", "return", "rsi", "macd", "macd_diff", "bollinger_pct"]
    for col in lag_cols:
        for lag in [1, 2, 3]:
            df_copy[f"{col}_lag{lag}"] = df_copy.groupby("ticker")[col].shift(lag)
    
    # Features cho Signal Model
    lags = [5, 10, 15]
    for lag in lags:
        df_copy[f'return_lag_{lag}'] = df_copy.groupby('ticker')['return'].shift(lag)

    vol_windows = [10, 20]
    for window in vol_windows:
        df_copy[f'volatility_{window}'] = df_copy.groupby('ticker')['return'].rolling(window=window).std().reset_index(0, drop=True)

    df_copy['rsi_momentum'] = df_copy.groupby('ticker')['rsi'].diff()
    df_copy['macd_diff_momentum'] = df_copy.groupby('ticker')['macd_diff'].diff()
    df_copy['price_vs_ema200'] = (df_copy['close'] / df_copy['ema_200']) - 1

    indicators_to_lag = ['rsi', 'mfi']
    for indicator in indicators_to_lag:
        for lag in [1, 3]:
            df_copy[f'{indicator}_lag_{lag}'] = df_copy.groupby('ticker')[indicator].shift(lag)

    return df_copy

In [23]:
class SignalBot:
    def __init__(self, tickers, timeframe, risk_model, signal_model,signal_model_params, notifier):
        self.tickers = tickers
        self.timeframe = timeframe
        self.risk_model = risk_model
        self.signal_model = signal_model
        self.notifier = TelegramNotifier(
            "8067634856:AAGoSKwgv7G_jDDLN73DJyTIkTPCdRZSAUc",  # BOT TOKEN
            "-1003002512643"  # CHAT ID group
        )

        # portfolio
        self.initial_balance = 1_000_000_000
        self.balance = self.initial_balance
        self.holdings = {}

        # params
        self.risk_per_trade_percent = 0.01
        self.initial_stop_loss_percent = 0.08
        self.trailing_stop_percent = 0.07
        self.max_hold_period = 20
        self.min_hold_period = 10

        # logs
        self.trade_log = []
        self.portfolio_history = []

    def _fetch_realtime_data(self, callback):
        event = client.Fetch_Trading_Data(
            realtime=True,
            tickers=self.tickers,
            fields=["open","high","low","close","volume"],
            adjusted=True,
            period=20,
            by=self.timeframe,
            callback=callback,
            wait_for_full_timeFrame=False
        )
        event.get_data()
        try:
            while not event._stop:
                time.sleep(1)
        except KeyboardInterrupt:
            print("⛔ Dừng realtime...")
            event.stop()

    def _process_realtime_data(self, data): # Đây chính là callback
        df = data.to_dataFrame()
        if len(df) < 2:
            return

        row = df.iloc[-2]  
        ticker = row["ticker"]

        hist_df = df[df["ticker"] == ticker].copy()

        self._process_bar(row, hist_df)

    def _process_bar(self, row, hist_df):
        # Xử lý 1 cây nến 
        ticker = row["ticker"]
        close_price = row["close"]

        # === Bộ lọc cơ bản ===
        df_today_filtered_raw = pd.DataFrame([row])

        df_today_filtered_raw['ema_50'] = fi.ema(df_today_filtered_raw['close'], window=50)
        df_today_filtered_raw['ema_200'] = fi.ema(df_today_filtered_raw['close'], window=200)
        df_today_filtered_raw['macd'] = fi.macd(df_today_filtered_raw['close'], window_fast=12, window_slow=26)
        df_today_filtered_raw['macd_signal'] = fi.macd_signal(df_today_filtered_raw['close'], window_fast=12, window_slow=26, window_sign=9)
        df_today_filtered_raw['macd_diff'] = fi.macd_diff(df_today_filtered_raw['close'], window_fast=12, window_slow=26, window_sign=9)
        df_today_filtered_raw['rsi'] = fi.rsi(df_today_filtered_raw['close'], window=14)
        df_today_filtered_raw['bollinger_hband'] = fi.bollinger_hband(df_today_filtered_raw['close'], window=20, window_dev=2)
        df_today_filtered_raw['bollinger_lband'] = fi.bollinger_lband(df_today_filtered_raw['close'], window=20, window_dev=2)
        df_today_filtered_raw['mfi'] = fi.mfi(df_today_filtered_raw['high'], df_today_filtered_raw['low'], df_today_filtered_raw['close'], df_today_filtered_raw['volume'], window=14)

        if self.list_qualified_tickers:
            if ticker not in self.list_qualified_tickers:
                return

        df_today_filtered_raw = add_features(df_today_filtered_raw)

        if list_qualified_tickers:
            df_today_filtered_raw = df_today_filtered_raw[
                df_today_filtered_raw["ticker"].isin(list_qualified_tickers)
            ]
        df_today_filtered_raw = filter_ticker_by_indicators(df_today_filtered_raw)
        if df_today_filtered_raw.empty:
            return


        # === Risk model ===
        risk_model_vnx = RiskClassifier()
        risk_model_vnx.load_model("D:\Downloads\DSTC vòng 3\weight\lgbm_model_vnx.pkl")
        risk_pred = risk_model_vnx.predict(df_today_filtered_raw)[0]

        if risk_pred != 0:
            return

        # === Signal model ===
        signal_model = SignalClassifier(**signal_model_params)
        input_size = len(signal_model.top_features)
        signal_model.load_model(input_size=input_size)

        hist_data = hist_df[hist_df["ticker"] == ticker].copy()
        hist_data['ema_50'] = fi.ema(hist_data['close'], window=50)
        hist_data['ema_200'] = fi.ema(hist_data['close'], window=200)
        hist_data['macd'] = fi.macd(hist_data['close'], window_fast=12, window_slow=26)
        hist_data['macd_signal'] = fi.macd_signal(hist_data['close'], window_fast=12, window_slow=26, window_sign=9)
        hist_data['macd_diff'] = fi.macd_diff(hist_data['close'], window_fast=12, window_slow=26, window_sign=9)
        hist_data['rsi'] = fi.rsi(hist_data['close'], window=14) 
        hist_data['bollinger_hband'] = fi.bollinger_hband(hist_data['close'], window=20, window_dev=2)
        hist_data['bollinger_lband'] = fi.bollinger_lband(dhist_data['close'], window=20, window_dev=2)
        hist_data['mfi'] = fi.mfi(hist_data['high'], hist_data['low'], hist_data['close'], hist_data['volume'], window=14) 
        hist_data['ema_200'] = np.where(hist_data['ema_200'].isna(), hist_data['ema_50'], hist_data['ema_200'])

        hist_data = add_features(histdata)
        hist_data = filter_ticker_by_indicators(hist_data)

        signal = signal_model.predict(hist_data)


        # =========== QUẢN LÝ HOLDINGS ===========
        if ticker in self.holdings:
            pos = self.holdings[ticker]
            pos["days_held"] += 1
            pos["high_since_entry"] = max(pos["high_since_entry"], close_price)
            pos["trailing_stop"] = pos["high_since_entry"] * (1 - self.trailing_stop_percent)

            should_sell = False
            reason = ""
            shares = pos["shares"]

            if close_price <= pos["initial_stop_price"]:
                should_sell, reason = True, "Stoploss"
            elif close_price <= pos["trailing_stop"]:
                should_sell, reason = True, "Trailing Stop"
            elif pos["days_held"] >= self.max_hold_period:
                should_sell, reason = True, f"Max Hold {self.max_hold_period}d"
            elif pos["days_held"] >= self.min_hold_period and signal == 0:
                should_sell, reason = True, "SignalClassifier SELL"

            if should_sell and shares > 0:
                self.balance += shares * close_price
                trade = {
                    "ticker": ticker,
                    "action": "SELL",
                    "price": close_price,
                    "shares": shares,
                    "reason": reason,
                }
                self.trade_log.append(trade)

                msg = (
                    f"📉 <b>BÁN</b>\n"
                    f"- Mã: {ticker}\n"
                    f"- Số lượng: {shares:,} cp\n"
                    f"- Giá bán: {close_price:,.0f}\n"
                    f"- Lý do: {reason}\n"
                    f"- Tiền mặt sau bán: {self.balance:,.0f} VND"
                )
                self.notifier.send_message(msg)
                del self.holdings[ticker]
                return

        # =========== TÍN HIỆU MUA ===========
        if signal == 2 and ticker not in self.holdings:
            stop_loss_price = close_price * (1 - self.initial_stop_loss_percent)
            risk_per_share = close_price - stop_loss_price
            capital_at_risk = self.balance * self.risk_per_trade_percent
            shares = int(capital_at_risk // risk_per_share)
            position_value = shares * close_price

            if shares > 0 and position_value <= self.balance:
                self.balance -= position_value
                self.holdings[ticker] = {
                    "shares": shares,
                    "entry_price": close_price,
                    "entry_date": row["timestamp"],
                    "high_since_entry": close_price,
                    "trailing_stop": close_price * (1 - self.trailing_stop_percent),
                    "initial_stop_price": stop_loss_price,
                    "days_held": 0,
                }
                trade = {
                    "ticker": ticker,
                    "action": "BUY",
                    "price": close_price,
                    "shares": shares,
                }
                self.trade_log.append(trade)

                msg = (
                    f"📈 <b>MUA</b>\n"
                    f"- Mã: {ticker}\n"
                    f"- Giá mua: {close_price:,.0f}\n"
                    f"- Stoploss: {stop_loss_price:,.0f}\n"
                    f"- Số lượng: {shares:,} cp\n"
                    f"- Tổng vốn: {position_value:,.0f} VND\n"
                    f"- Tiền mặt còn lại: {self.balance:,.0f} VND"
                )
                self.notifier.send_message(msg)

    def run_backtest(self,all_df, RiskClassifier, SignalClassifier, vnindex_df, 
                    risk_model_params={},
                    signal_model_params={},
                    initial_balance=1_000_000_000,
                    trailing_stop_percent=0.07,
                    initial_stop_loss_percent=0.08,
                    min_hold_period=10,
                    max_hold_period=20,
                    full_take_profit_percent=0.20,
                    partial_take_profit_percent = 0.10,
                    partial_take_profit_amount = 0.5,
                    max_concurrent_positions=15,  
                ):
        # --- B0: Feature Engineering ---
        df_vnx, df_hnx, df_upcom = create_all_features(all_df)
        all_df_featured = pd.concat([df_vnx, df_hnx, df_upcom], ignore_index= True)
        all_df_featured.dropna(inplace=True)
        all_df_featured = all_df_featured.sort_values("timestamp").set_index("timestamp")

        # --- Chia train/test ---
        unique_dates = all_df_featured.index.unique().sort_values()
        split_idx = int(0.80 * len(unique_dates))
        train_cutoff = unique_dates[split_idx]
        train_df = all_df_featured.loc[all_df_featured.index <= train_cutoff]
        test_df_raw = all_df_featured.loc[all_df_featured.index > train_cutoff]

        dates_to_drop = [
            '2025-05-08', '2025-06-05', '2025-06-12', '2025-06-17',
            '2025-06-25', '2025-06-30', '2025-07-03', '2025-01-03',
            '2025-04-21', '2025-01-20', '2025-04-24', '2025-01-16'
        ]
        
        test_df_dates_str = pd.to_datetime(test_df_raw.index).strftime('%Y-%m-%d')
        
        test_df = test_df_raw[~np.isin(test_df_dates_str, dates_to_drop)]

        risk_model_vnx = RiskClassifier()
        risk_model_hnx = RiskClassifier()
        risk_model_upcom = RiskClassifier()

        risk_model_vnx.load_model("D:\Downloads\DSTC vòng 3\weight\lgbm_model_vnx.pkl")
        risk_model_hnx.load_model("D:\Downloads\DSTC vòng 3\weight\lgbm_model_hnx.pkl")
        risk_model_upcom.load_model("D:\Downloads\DSTC vòng 3\weight\lgbm_model_upcom.pkl")

        # --- Load Signal model ---
        signal_model = SignalClassifier(**signal_model_params)
        input_size = len(signal_model.top_features)
        signal_model.load_model(input_size=input_size)

        all_trading_days = test_df.index.unique().sort_values()    
        trading_days = all_trading_days[:-3] 
        holdings = {}
        print("\n🚀 BẮT ĐẦU BACKTEST TRÊN TEST_DF 🚀")

        for date in tqdm(trading_days, desc="Backtesting"):    
            df_today_full_raw = test_df.loc[[date]] 
            df_today_full = df_today_full_raw.set_index('ticker')
            df_today_filtered_raw = df_today_full_raw.copy() 
            if list_qualified_tickers:
                df_today_filtered_raw = df_today_filtered_raw[df_today_filtered_raw["ticker"].isin(list_qualified_tickers)]
            df_today_filtered_raw = filter_ticker_by_indicators(df_today_filtered_raw)
            df_today = df_today_filtered_raw.set_index('ticker')
        
            is_market_bullish = True 
            if date in vnindex_df.index and not np.isnan(vnindex_df.loc[date, 'sma200']):
                is_market_bullish = vnindex_df.loc[date, 'close'] > vnindex_df.loc[date, 'sma200']

            # --- 1. Risk & Signal Classification ---
            risk_preds = []

            mask_hose = df_today["exchange"] == "HOSE"
            if mask_hose.any():
                preds_hose = risk_model_vnx.predict(df_today.loc[mask_hose])
                risk_preds.extend(list(zip(df_today.loc[mask_hose].index, preds_hose)))

            mask_hnx = df_today["exchange"] == "HNX"
            if mask_hnx.any():
                preds_hnx = risk_model_hnx.predict(df_today.loc[mask_hnx])
                risk_preds.extend(list(zip(df_today.loc[mask_hnx].index, preds_hnx)))

            mask_upcom = df_today["exchange"] == "UPCOM"
            if mask_upcom.any():
                preds_upcom = risk_model_upcom.predict(df_today.loc[mask_upcom])
                risk_preds.extend(list(zip(df_today.loc[mask_upcom].index, preds_upcom)))

            risk_preds_series = pd.Series(dict(risk_preds))

            eligible_tickers = risk_preds_series[risk_preds_series == 0].index.tolist()
            if eligible_tickers:
                print(f"\n[{date}] Các mã đủ điều kiện: {len(eligible_tickers)} - {eligible_tickers}")
            
            signals = {}
            if eligible_tickers:
                for ticker in eligible_tickers:
                    hist_data = all_df_featured[all_df_featured["ticker"] == ticker].loc[:date].copy()
                    if not hist_data.empty:
                        signal = signal_model.predict(hist_data)
                        signals[ticker] = signal
            
            buy_signals = {t: s for t, s in signals.items() if s == 2}
            sell_signals = {t: s for t, s in signals.items() if s == 0}
            
            # --- 2. Quản lý Holdings ---
            current_holdings = list(holdings.keys())
            for ticker in current_holdings:
                if ticker in df_today_full.index:
                    current_price = df_today_full.loc[ticker, "close"]
                    entry_price = holdings[ticker]['entry_price']
                    
                    holdings[ticker]['high_since_entry'] = max(holdings[ticker]['high_since_entry'], current_price)
                    holdings[ticker]['trailing_stop'] = holdings[ticker]['high_since_entry'] * (1 - trailing_stop_percent)
                    holdings[ticker]['days_held'] += 1

                    should_sell = False
                    reason_to_sell = ""

                    current_profit_percent = (current_price / entry_price) - 1
                    if not holdings[ticker]['partial_profit_taken']:
                        if current_profit_percent >= partial_take_profit_percent:
                            shares_to_sell = int(holdings[ticker]['shares'] * partial_take_profit_amount) 
                            if shares_to_sell > 0:
                                self.balance += shares_to_sell * current_price
                                holdings[ticker]['shares'] -= shares_to_sell 
                                holdings[ticker]['partial_profit_taken'] = True
                                reason_to_sell = "Chốt lời 1/2"

                                msg = (
                                    f"📉 <b>BÁN</b>\n"
                                    f"- Ngày: {date}\n"
                                    f"- Mã: {ticker}\n"
                                    f"- Giá: {current_price:,.0f}\n"
                                    f"- Số lượng: {shares_to_sell:,} cp\n"
                                    f"- Lý do: {reason_to_sell}\n"
                                    f"- Tiền mặt: {self.balance:,.0f} VND"
                                )
                                self.notifier.send_message(msg)
                                
                    if current_profit_percent >= full_take_profit_percent:
                        should_sell = True
                        reason_to_sell = f"Take Profit ({full_take_profit_percent:.0%})"
                    

                    elif current_price <= holdings[ticker].get('initial_stop_price', 0):
                        should_sell = True
                        reason_to_sell = "Initial Stop-loss"
                        
                    elif current_price <= holdings[ticker]['trailing_stop']:
                        should_sell = True
                        reason_to_sell = "Trailing Stop"
                        
                    elif holdings[ticker]['days_held'] >= min_hold_period:
                        if signals.get(ticker, 1) == 0:
                            should_sell = True
                            reason_to_sell = "SignalClassifier BÁN"
                        elif holdings[ticker]['days_held'] >= max_hold_period:
                            should_sell = True
                            reason_to_sell = f"Max Hold Period ({max_hold_period} days)"
                            
                    if should_sell and holdings[ticker]['shares'] > 0:
                        self.balance += holdings[ticker]['shares'] * current_price

                        print(f"  -> BÁN TOÀN BỘ: {holdings[ticker]['shares']} {ticker} tại giá {current_price:,.0f} ({reason_to_sell})")
                        shares = holdings[ticker]['shares']
                        
                        del holdings[ticker]
                        msg = (
                            f"📉 <b>BÁN</b>\n"
                            f"- Ngày: {date}\n"
                            f"- Mã: {ticker}\n"
                            f"- Giá: {current_price:,.0f}\n"
                            f"- Số lượng: {shares:,} cp\n"
                            f"- Lý do: {reason_to_sell}\n"
                            f"- Tiền mặt: {self.balance:,.0f} VND"
                        )
                        self.notifier.send_message(msg)

            buy_tickers_today = [t for t, s in buy_signals.items() if t not in holdings]

            if buy_tickers_today and is_market_bullish:
                slots_available = max_concurrent_positions - len(holdings)
                
                if slots_available <= 0:
                    print("  -> Đã đạt giới hạn số lượng vị thế. Không mua mới.")
                else:
                    buy_tickers_to_consider = buy_tickers_today[:slots_available]
                    
                    risk_per_trade_percent = 0.01 
                    initial_stop_loss_percent = 0.08
                    max_capital_deployment_per_day = 0.40
                    
                    # Tính tổng giá trị tài sản hiện tại (tiền mặt + giá trị cổ phiếu)
                    holdings_value = 0
                    for ticker_held, info_held in holdings.items():
                        if ticker_held in df_today_full.index:
                            holdings_value += info_held['shares'] * df_today_full.loc[ticker_held, "close"]
                    total_portfolio_value = self.balance + holdings_value
                    
                    # Tính toán tổng vốn có thể triển khai trong ngày hôm nay
                    capital_to_deploy_today = total_portfolio_value * max_capital_deployment_per_day
                
                    
                    for ticker in buy_tickers_to_consider: 
                        if capital_to_deploy_today <= 0:
                            break 
            
                        if ticker in df_today_full.index:
                            price = df_today_full.loc[ticker, "close"]
                            
                            stop_loss_price = price * (1 - initial_stop_loss_percent)
                            
                            risk_per_share = price - stop_loss_price
            
                            if price > 0 and risk_per_share > 0:
                                capital_at_risk = total_portfolio_value * risk_per_trade_percent
                                
                                shares = int(capital_at_risk // risk_per_share)
                                
                                position_value = shares * price
                                
                                if shares > 0 and position_value <= self.balance and position_value <= capital_to_deploy_today:
                                    self.balance -= position_value
                                    capital_to_deploy_today -= position_value 
                                    
                                    holdings[ticker] = {
                                        'shares': shares, 
                                        'entry_price': price, 
                                        'entry_date': date, 
                                        'high_since_entry': price,
                                        'trailing_stop': price * (1 - trailing_stop_percent),
                                        'initial_stop_price': stop_loss_price,
                                        'days_held': 0, 
                                        'partial_profit_taken': False, 
                                        'status': 'initial'
                                    }
                                    print(f"  -> MUA: {shares} {ticker} tại giá {price:,.0f} (Vốn: {position_value:,.0f} VNĐ, Rủi ro: {capital_at_risk:,.0f} VNĐ)")
                                    msg = (
                                        f"📈 <b>MUA</b>\n"
                                        f"- Rủi ro dự kiến: {capital_at_risk:,.0f} VND"
                                        f"- Ngày: {date}\n"
                                        f"- Mã: {ticker}\n"
                                        f"- Giá: {price:,.0f}\n"
                                        f"- Số lượng: {shares:,} cp\n"
                                        f"- Tổng vốn: {position_value:,.0f} VND\n"
                                        f"- Tiền mặt còn lại: {self.balance:,.0f} VND"
                                    )     
                                    self.notifier.send_message(msg)


In [25]:
DATA_PATH_UPCOM = r"D:\Downloads\DSTC vòng 3\cleaned data\UPCOM_cleaned.xlsx"
DATA_PATH_HOSE = r"D:\Downloads\DSTC vòng 3\cleaned data\VNINDEX_cleaned.xlsx"
DATA_PATH_HNX = r"D:\Downloads\DSTC vòng 3\cleaned data\HNXINDEX_cleaned.xlsx"
MODEL_PATH = r'D:\Downloads\DSTC vòng 3\weight\best_lstm_classifier_improved.pth'
SCALER_PATH = r"D:\Downloads\DSTC vòng 3\weight\feature_scaler.pkl"


data_upcom = pd.read_excel(DATA_PATH_UPCOM)
data_hose = pd.read_excel(DATA_PATH_HOSE)
data_hnx = pd.read_excel(DATA_PATH_HNX)
data_hose["exchange"] = "HOSE"
data_hnx["exchange"] = "HNX"
data_upcom["exchange"] = "UPCOM"
data_full = pd.concat([data_upcom, data_hose, data_hnx], ignore_index= True)

if data_full is not None:
    signal_model_params = {
        "top_features": ['price_vs_ema200', 'volatility_20', 'volume', 'macd_diff', 'bollinger_bw', 'ema_200', 'volatility_10', 'macd_signal', 'rsi', 'bollinger_pct', 'macd', 'ema_50', 'bollinger_lband', 'mfi_lag_3', 'bollinger_hband'],
        "time_steps": 10, "device": "cpu", "model_path": MODEL_PATH, "scaler_path": SCALER_PATH
    }



    # Tạo bot
    bot = SignalBot(
        tickers=None,   # backtest không cần tickers
        timeframe="1d",
        risk_model=RiskClassifier,
        signal_model=SignalClassifier,
        signal_model_params=signal_model_params,
        notifier=TelegramNotifier(
                "8067634856:AAGoSKwgv7G_jDDLN73DJyTIkTPCdRZSAUc",  # BOT TOKEN
                "-1003002512643"  # CHAT ID group
            )
    )

    bot.run_backtest(
        all_df=data_full,
        RiskClassifier=RiskClassifier,
        vnindex_df = data_hose,
        SignalClassifier=SignalClassifier,
        signal_model_params=signal_model_params,
        initial_balance=1_000_000_000 
    )

[Signal Model] Load weight từ D:\Downloads\DSTC vòng 3\weight\best_lstm_classifier_improved.pth thành công
[Signal Model] Load scaler từ D:\Downloads\DSTC vòng 3\weight\feature_scaler.pkl thành công

🚀 BẮT ĐẦU BACKTEST TRÊN TEST_DF 🚀


Backtesting:   0%|          | 0/151 [00:00<?, ?it/s]


[2024-12-25 00:00] Các mã đủ điều kiện: 28 - ['BVS', 'VCS', 'NAG', 'L18', 'TIG', 'L14', 'IDV', 'MBS', 'NTP', 'IDJ', 'VC2', 'IDC', 'VC3', 'SCG', 'DXP', 'PBC', 'QNS', 'HD6', 'DRI', 'HND', 'ILA', 'FOX', 'DVN', 'AMS', 'SCL', 'AFX', 'GHC', 'NED']
  -> MUA: 9124 TIG tại giá 13,700 (Vốn: 124,998,800 VNĐ, Rủi ro: 10,000,000 VNĐ)
📩 Gửi Telegram thành công: {'ok': True, 'result': {'message_id': 115, 'from': {'id': 8067634856, 'is_bot': True, 'first_name': 'Stock Market DSTC', 'username': 'StockerCBot'}, 'chat': {'id': -1003002512643, 'title': 'Stock Market', 'is_forum': True, 'type': 'supergroup'}, 'date': 1758464712, 'text': '📈 MUA\n- Rủi ro dự kiến: 10,000,000 VND- Ngày: 2024-12-25 00:00\n- Mã: TIG\n- Giá: 13,700\n- Số lượng: 9,124 cp\n- Tổng vốn: 124,998,800 VND\n- Tiền mặt còn lại: 875,001,200 VND', 'entities': [{'offset': 3, 'length': 3, 'type': 'bold'}]}}
  -> MUA: 2383 NTP tại giá 52,445 (Vốn: 124,976,923 VNĐ, Rủi ro: 10,000,000 VNĐ)


Backtesting:   1%|          | 1/151 [00:03<08:49,  3.53s/it]

📩 Gửi Telegram thành công: {'ok': True, 'result': {'message_id': 116, 'from': {'id': 8067634856, 'is_bot': True, 'first_name': 'Stock Market DSTC', 'username': 'StockerCBot'}, 'chat': {'id': -1003002512643, 'title': 'Stock Market', 'is_forum': True, 'type': 'supergroup'}, 'date': 1758464713, 'text': '📈 MUA\n- Rủi ro dự kiến: 10,000,000 VND- Ngày: 2024-12-25 00:00\n- Mã: NTP\n- Giá: 52,445\n- Số lượng: 2,383 cp\n- Tổng vốn: 124,976,923 VND\n- Tiền mặt còn lại: 750,024,277 VND', 'entities': [{'offset': 3, 'length': 3, 'type': 'bold'}]}}

[2024-12-26 00:00] Các mã đủ điều kiện: 28 - ['IDC', 'BVS', 'VHE', 'DXP', 'IDV', 'SD9', 'MBS', 'VC2', 'NAG', 'SCG', 'TNG', 'NTP', 'TIG', 'DTD', 'SSH', 'ACV', 'SCL', 'AFX', 'FOX', 'GHC', 'DVN', 'HND', 'DDV', 'NED', 'HD6', 'PBC', 'DRI', 'ILA']
  -> MUA: 16195 AFX tại giá 7,711 (Vốn: 124,879,645 VNĐ, Rủi ro: 9,990,876 VNĐ)
📩 Gửi Telegram thành công: {'ok': True, 'result': {'message_id': 117, 'from': {'id': 8067634856, 'is_bot': True, 'first_name': 'Stock Ma

...

📩 Gửi Telegram thành công: {'ok': True, 'result': {'message_id': 219, 'from': {'id': 8067634856, 'is_bot': True, 'first_name': 'Stock Market DSTC', 'username': 'StockerCBot'}, 'chat': {'id': -1003002512643, 'title': 'Stock Market', 'is_forum': True, 'type': 'supergroup'}, 'date': 1758465024, 'text': '📈 MUA\n- Rủi ro dự kiến: 13,427,790 VND- Ngày: 2025-08-20 00:00\n- Mã: DTD\n- Giá: 21,700\n- Số lượng: 7,734 cp\n- Tổng vốn: 167,827,800 VND\n- Tiền mặt còn lại: 28,146,944 VND', 'entities': [{'offset': 3, 'length': 3, 'type': 'bold'}]}}

[2025-08-21 00:00] Các mã đủ điều kiện: 24 - ['VC3', 'L18', 'MBG', 'C69', 'SD9', 'VCS', 'VC2', 'DXP', 'CAP', 'HBS', 'VHE', 'CEO', 'DTD', 'VLB', 'HD6', 'SGP', 'ABI', 'GHC', 'DVN', 'C4G', 'NED', 'HND', 'CLX', 'NTC']


Backtesting:  99%|█████████▉| 150/151 [05:15<00:02,  2.28s/it]


[2025-08-22 00:00] Các mã đủ điều kiện: 22 - ['SHS', 'HBS', 'MBS', 'VHE', 'VC2', 'PVI', 'C69', 'SD9', 'VC3', 'NAG', 'MBG', 'TIG', 'SCG', 'IVS', 'NED', 'GHC', 'HU4', 'C4G', 'HND', 'ABI', 'AMS', 'QTP']
  -> BÁN TOÀN BỘ: 14434 DXP tại giá 10,800 (Trailing Stop)
❌ Lỗi gửi Telegram: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
  -> BÁN TOÀN BỘ: 13048 DRI tại giá 11,796 (Initial Stop-loss)
📩 Gửi Telegram thành công: {'ok': True, 'result': {'message_id': 220, 'from': {'id': 8067634856, 'is_bot': True, 'first_name': 'Stock Market DSTC', 'username': 'StockerCBot'}, 'chat': {'id': -1003002512643, 'title': 'Stock Market', 'is_forum': True, 'type': 'supergroup'}, 'date': 1758465027, 'text': '📉 BÁN\n- Ngày: 2025-08-22 00:00\n- Mã: DRI\n- Giá: 11,796\n- Số lượng: 13,048 cp\n- Lý do: Initial Stop-loss\n- Tiền mặt: 337,948,352 VND', 'entities': [{'offset': 3, 'length': 3, 'type': 'bold'}]}}
  -> BÁN TOÀN BỘ: 7

Backtesting: 100%|██████████| 151/151 [05:21<00:00,  2.13s/it]

📩 Gửi Telegram thành công: {'ok': True, 'result': {'message_id': 223, 'from': {'id': 8067634856, 'is_bot': True, 'first_name': 'Stock Market DSTC', 'username': 'StockerCBot'}, 'chat': {'id': -1003002512643, 'title': 'Stock Market', 'is_forum': True, 'type': 'supergroup'}, 'date': 1758465031, 'text': '📈 MUA\n- Rủi ro dự kiến: 12,933,252 VND- Ngày: 2025-08-22 00:00\n- Mã: TIG\n- Giá: 10,200\n- Số lượng: 15,849 cp\n- Tổng vốn: 161,659,800 VND\n- Tiền mặt còn lại: 9,202,652 VND', 'entities': [{'offset': 3, 'length': 3, 'type': 'bold'}]}}
